# Notes

Data processing from "raw" json

data to be split by excersise

all pois shall be
- without rebase
    - normalised globaly
    - normalised regionaly
    - distances between time steps
    - directional movment between time steps
- with rebase
    - normalised globaly with rebase on middle base (nose)
    - normalised regionlay with regional rebase
    - distances between time steps
    - directional movment between time steps

In [1]:
import os
import json
import math

import numpy as np


# Constants

In [2]:
dir_data_root = os.path.join('data')
dir_data_source = os.path.join(dir_data_root, 'json')
dir_data_target = os.path.join(dir_data_root, 'json_preprocessed')

# Data prep

In [3]:
pois_order = {
    '0_LefteyeMidbottom': 0,
    '1_LefteyeMidtop': 1,
    '2_LefteyeInnercorner': 2,
    '3_LefteyeOutercorner': 3,  
    '6_RighteyeMidbottom': 4,  
    '7_RighteyeMidtop': 5,
    '8_RighteyeInnercorner': 6,  
    '9_RighteyeOutercorner': 7,
    '4_LefteyebrowInner': 8,
    '5_LefteyebrowCenter': 8,
    '18_ForeheadCenter': 10,
    '10_RighteyebrowInner': 11,
    '11_RighteyebrowCenter': 12,
    '12_NoseTip': 13,
    '13_MouthLowerlipMidbottom': 14,
    '14_MouthLeftcorner': 15,
    '15_MouthRightcorner': 16,
    '16_MouthUpperlipMidtop': 17,
    '17_ChinCenter': 18,
    '19_LeftcheekCenter': 19,
    '20_RightcheekCenter': 20,
}

In [4]:
def rescale(value, minimum, maximum):
    return (value - minimum) / (maximum - minimum)

In [5]:
def normalise(xs):
    xs_minimum = min(xs)
    xs_maximum = max(xs)
    
    return [rescale(x, xs_minimum, xs_maximum) for x in xs]

In [6]:
def distance(xs, ys, zs):
    distance = [0]
    for i in range(1, len(xs)):
        distance.append(math.dist([xs[i-1], ys[i-1], zs[i-1]], [xs[i], ys[i], zs[i]]))
    
    return distance

In [7]:
def rebase(xs, base_xs):
    return[x - bx for x,bx in zip(xs, base_xs)]

In [8]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

In [9]:
def angle_between_two_points(p1, p2):
    p1_u = unit_vector(p1)
    p2_u = unit_vector(p2)
    return np.arccos(np.clip(np.dot(p1_u, p2_u), -1.0, 1.0))

In [10]:
def direction_angles(xs, ys, zs):
    angles = [0]
    for i in range(1, len(xs)):
        angles.append(angle_between_two_points([xs[i-1], ys[i-1], zs[i-1]], [xs[i], ys[i], zs[i]]))
    
    return angles


In [11]:
def key_pois(poi):
    return pois_order[poi['meta']['tag']]

Read every file in the target directory and apply mapper function

In [12]:
amount_excercises = 0

for file_name in os.listdir(dir_data_source):
    with open(os.path.join(dir_data_source, file_name)) as f:
       
        patient_id = file_name.split('.')[0]
        excercise_number = 0
        print('Loading raw data for patiend: %s' % patient_id)
        
        data = json.load(f)
        
        for session in data:
            for excercise in session['exercises']:
                
                excercise_number += 1
                
                excercise_meta = {
                    'patient_id': session['meta']['patient_id'],
                    'exercise_dates': session['meta']['exercise_dates'],
                    'evaluation': session['meta']['evaluation'],
                    'flag_before_surgery': session['meta']['flag_before_surgery'],
                    'excercise_tag': excercise['meta']['tag'],
                    'excercise_id': excercise['meta']['id'],
                    'excercise_name': excercise['meta']['name']
                }
                
                processed_excercise = {
                    'meta': excercise_meta,
                    'regions': {}
                }
                
                base = {}
                for poi in excercise['pois']:
                    if poi['meta']['base'] == 'base':
                        base = {
                            'xs': poi['input']['xs'],
                            'ys': poi['input']['ys'],
                            'zs': poi['input']['zs'],
                        }
                        break
                        
                global_region = []
                orbital_region = []
                frontal_region = []
                oral_region = []
                global_region_rebased = []
                orbital_region_rebased = []
                frontal_region_rebased = []
                oral_region_rebased = []
                
                pois_sorted = sorted(excercise['pois'], key = key_pois)
                for poi in pois_sorted:
                    xs = normalise(poi['input']['xs'])
                    ys = normalise(poi['input']['ys'])
                    zs = normalise(poi['input']['zs'])
                    distances = distance(xs, ys, zs)
                    #directions = direction_angles(xs, ys, zs)
                    
                    global_region.append(xs)
                    global_region.append(ys)
                    global_region.append(zs)
                    global_region.append(distances)
                    #global_region.append(directions)
                    
                    if poi['meta']['region'] == 'orbital':
                        orbital_region.append(xs)
                        orbital_region.append(ys)
                        orbital_region.append(zs)
                        orbital_region.append(distances)
                        #orbital_region.append(directions)
                    if poi['meta']['region'] == 'frontal':
                        frontal_region.append(xs)
                        frontal_region.append(ys)
                        frontal_region.append(zs)
                        frontal_region.append(distances)
                        #frontal_region.append(directions) 
                    if poi['meta']['region'] == 'oral':
                        oral_region.append(xs)
                        oral_region.append(ys)
                        oral_region.append(zs)
                        oral_region.append(distances)
                        #oral_region.append(directions)    
                    
                    processed_excercise['regions']['global'] = global_region
                    processed_excercise['regions']['orbital'] = orbital_region
                    processed_excercise['regions']['frontal'] = frontal_region
                    processed_excercise['regions']['oral'] = oral_region
                    
                    #TODO start here        
                    #TODO add rebase and normalise
                    if poi['meta']['base'] == 'base':
                        continue
                                            
                    xs_rebased = normalise(rebase(poi['input']['xs'], base['xs']))
                    ys_rebased = normalise(rebase(poi['input']['ys'], base['ys']))
                    zs_rebased = normalise(rebase(poi['input']['zs'], base['zs']))
                    distances_rebased = distance(xs_rebased, ys_rebased, zs_rebased)
                    
                    global_region_rebased.append(xs_rebased)
                    global_region_rebased.append(ys_rebased)
                    global_region_rebased.append(zs_rebased)
                    global_region_rebased.append(distances_rebased)
                    
                    if poi['meta']['region'] == 'orbital':
                        orbital_region_rebased.append(xs_rebased)
                        orbital_region_rebased.append(ys_rebased)
                        orbital_region_rebased.append(zs_rebased)
                        orbital_region_rebased.append(distances_rebased)
                        #orbital_region.append(directions)
                    if poi['meta']['region'] == 'frontal':
                        frontal_region_rebased.append(xs_rebased)
                        frontal_region_rebased.append(ys_rebased)
                        frontal_region_rebased.append(zs_rebased)
                        frontal_region_rebased.append(distances_rebased)
                        #frontal_region.append(directions) 
                    if poi['meta']['region'] == 'oral':
                        oral_region_rebased.append(xs_rebased)
                        oral_region_rebased.append(ys_rebased)
                        oral_region_rebased.append(zs_rebased)
                        oral_region_rebased.append(distances_rebased)
                        #oral_region.append(directions)  
                
                processed_excercise['regions']['global_rebased'] = global_region_rebased
                processed_excercise['regions']['orbital_rebased'] = orbital_region_rebased
                processed_excercise['regions']['frontal_rebased'] = frontal_region_rebased
                processed_excercise['regions']['oral_rebased'] = oral_region_rebased
                
                with open(os.path.join(dir_data_target, '%s_%03d.json' % (patient_id, excercise_number)), "w") as f_w:
                    json.dump(processed_excercise, f_w)
                
    print('Processed %03d excersises' % excercise_number)

Loading raw data for patiend: 00000000082
Processed 009 excersises
Loading raw data for patiend: 00000000057
Processed 009 excersises
Loading raw data for patiend: 00000000000
Processed 002 excersises
Loading raw data for patiend: 00000000016
Processed 009 excersises
Loading raw data for patiend: 00000000041
Processed 018 excersises
Loading raw data for patiend: 00000000036
Processed 009 excersises
Loading raw data for patiend: 00000000061
Processed 018 excersises
Loading raw data for patiend: 00000000077
Processed 009 excersises
Loading raw data for patiend: 00000000020
Processed 018 excersises
Loading raw data for patiend: 00000000076
Processed 009 excersises
Loading raw data for patiend: 00000000060
Processed 009 excersises
Loading raw data for patiend: 00000000037
Processed 009 excersises
Loading raw data for patiend: 00000000040
Processed 009 excersises
Loading raw data for patiend: 00000000017
Processed 036 excersises
Loading raw data for patiend: 00000000001
Processed 027 excers

## Data set per session

## Input shape

## Output shape

__END__